In [1]:
import scipy.special as sp
import numpy as np
import uproot
import csv

import Utilities.Constants as Constants
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print("Success")

Success


## Purpose

This notebook was written to try and calculate a very rough "back of the envelope" observed limit just using a one-bin counting experiment. <br>
As it stands, I was not able to add the signal uncertainty into this and so it gives a very optimistic limit. 

In [3]:
frac = 0.1
frac_1_sig = 0.317
upper = -1*np.log(frac)
upper_1_sig = -1*np.log(frac_1_sig)
print(upper)
print(upper_1_sig)

2.3025850929940455
1.1488535051048565


## Loading histograms

In [3]:
Params = {"Use_flat_sys_bkg":True,
               "Use_flat_sys_signal":True,
               "Stats_only":False,
               "Use_second_half_only":True,
               "Load_logit_hists":True,
               "Use_toys":True,
               "Num_toys":100,
               "Load_lepton_hists":True,
               "Load_pi0_hists":False,
               "Flat_overlay_bkg_frac":0.5,
               "Flat_dirt_bkg_frac":1.0,
               "Flat_sig_frac":0.5,
               "Signal_flux_error":0.3}

Functions.pyhf_params(Params)

Using FLAT systematic uncertainty on background
With 50.0% on overlay, and 100.0% on dirt.
Using FLAT systematic uncertainty on signal
With 50.0% on all signal


In [15]:
HNL_mass = 150
hist_file = uproot.open(f'bdt_output/pi0/run1_{HNL_mass}MeV_logit_test1.root')
theta_squared = Constants.theta_mu_4*Constants.theta_mu_4
print("Theta squared is " + str(theta_squared))

signal_hist = hist_file['signal']
overlay_hist = hist_file['bkg_overlay']
EXT_hist = hist_file['bkg_EXT']
dirt_hist = hist_file['bkg_dirt']
bkg_hists = [overlay_hist, EXT_hist, dirt_hist]

total_bkg = Functions.add_hists_vals(bkg_hists)

#Errors, should have this saved in the initial .root file (i.e do at the end of 3.5)
bkg_stat_err_list = [overlay_hist.errors(), 
                     EXT_hist.errors(), 
                     dirt_hist.errors()]

bkg_sys_err_list = [overlay_hist.values()*Params["Flat_overlay_bkg_frac"], 
                    np.zeros_like(EXT_hist.errors()), #No systematic error on the EXT sample
                    dirt_hist.values()*Params["Flat_dirt_bkg_frac"]]

bkg_err_list = [Functions.add_all_errors([bkg_stat_err_list[0],bkg_sys_err_list[0]]), #adding the sys and stat error in quadrature for each bkg type
                Functions.add_all_errors([bkg_stat_err_list[1],bkg_sys_err_list[1]]),
                Functions.add_all_errors([bkg_stat_err_list[2],bkg_sys_err_list[2]])]

total_bkg_err = Functions.add_all_errors(bkg_err_list)



Theta squared is 1e-08


In [13]:
Num_bkg = total_bkg[-1]
Err_bkg = total_bkg_err[-1]
Num_signal = signal_hist.values()[-1]

print("Number of events in final bin is " + str(Num_bkg))
print("Error on that is " + str(Err_bkg))
print("Number of signal events is " + str(Num_signal))

Number of events in final bin is 0.27213806
Error on that is 0.31010061702930525
Number of signal events is 0.0037438546


In [16]:
CL = 0.9 #0.9 for 90% CL

Num_signal_allowed = sp.erfinv(2*(CL-0.5))*np.sqrt(2)*(Err_bkg)
print("Number of signal events allowed at 90% CL is " + str(Num_signal_allowed))

scaling = Num_signal_allowed/Num_signal

print("Need to scale signal number by " + str(scaling))

EXP_LIMIT = np.sqrt(scaling)*theta_squared

print("Expected limit is " + str(EXP_LIMIT))

Number of signal events allowed at 90% CL is 0.39740993123025276
Need to scale signal number by 106.1499377968244
Expected limit is 1.0302909190943323e-07
